In [1]:
import sys
sys.path.append('../../laddie/src/')

import numpy as np
import xarray as xr
import pandas as pd
import os
import copy
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo

In [3]:
# Read in original ocean3 and ocean4 data
OC3 = xr.open_dataset('original_data/Ocean3_input_geom_v1.01.nc')
OC4 = xr.open_dataset('original_data/Ocean4_input_geom_v1.01.nc')

# Compute thickness, mask and rename surface, bed
OC3_2 = OC3.rename(name_dict={'bedrockTopography':'bed', 'upperSurface':'surface', 'lowerSurface':'draft'})
OC3_2['mask'] = xr.where(OC3.floatingMask==1,3,np.nan)
OC3_2['mask'] = xr.where(OC3.openOceanMask==1,0,OC3_2.mask)
OC3_2['mask'] = xr.where(OC3.groundedMask==1,2,OC3_2.mask)
OC3_2['thickness'] = xr.where(OC3_2.mask>0, OC3_2.surface - OC3_2.draft, np.nan)
OC3_new = OC3_2.drop_vars(["openOceanMask", "groundedMask","floatingMask"])

OC4_2 = OC4.rename(name_dict={'bedrockTopography':'bed', 'upperSurface':'surface', 'lowerSurface':'draft'})
OC4_2['mask'] = xr.where(OC4.floatingMask==1,3,np.nan)
OC4_2['mask'] = xr.where(OC4.openOceanMask==1,0,OC4_2.mask)
OC4_2['mask'] = xr.where(OC4.groundedMask==1,2,OC4_2.mask)
OC4_2['thickness'] = xr.where(OC4_2.mask>0, OC4_2.surface - OC4_2.draft, np.nan)
OC4_new = OC4_2.drop_vars(["openOceanMask", "groundedMask","floatingMask"])

# Add grounded ice east of domain to prevent cyclic boundary conditions
boarder = OC3_new.isel(y=[78,79])
boarders = boarder.assign_coords({"y": boarder.y + 2000})
boarders['mask'] = xr.where(np.logical_or(boarders.mask == 0,boarders.mask ==3), 2, boarders.mask)
OC3_boarder = xr.concat([OC3_new,boarders],dim='y')

boarder = OC4_new.isel(y=[78,79])
boarders = boarder.assign_coords({"y": boarder.y + 2000})
boarders['mask'] = xr.where(np.logical_or(boarders.mask == 0,boarders.mask ==3), 2, boarders.mask)
OC4_boarder = xr.concat([OC4_new,boarders],dim='y')

# Save new files to data folder
OC3_boarder.to_netcdf('OC3.nc')
OC4_boarder.to_netcdf('OC4.nc')